In [1]:
from ltp import LTP
from itertools import chain
from collections import defaultdict

def build_T2H(dep):
    '''
    根据依存树构建每个词的发射字典
    txt: 设备机房、电梯机房、水箱间、天线
    dep: [
        (1, 2, 'ATT'), (2, 0, 'HED'), (3, 5, 'WP'), 
        (4, 5, 'ATT'), (5, 2, 'COO'), (6, 7, 'WP'), 
        (7, 2, 'COO'), (8, 9, 'WP'), (9, 2, 'COO')
    ]
    return {
        2: {'ATT': [1], 'COO': [5, 7, 9]}, 
        0: {'HED': [2]}, 
        5: {'ATT': [4]}
    }
    '''
    dep_T2H = defaultdict(dict)
    for d in dep:
        # if d[2] in ['WP','LAD']: continue
        if d[1] in dep_T2H:
            dep_T2H[d[1]][d[2]] = dep_T2H[d[1]].get(d[2],[])+[d[0]]
        else:
            dep_T2H[d[1]] = {d[2]:[d[0]]}
    return dep_T2H
    
def find_smallest(dep_T2H, p):
    '''
    寻找当前中心语的修饰覆盖的范围
    由于不存在交叉的情况，所以只要往前找最小的即可
    return: 第一个词对应的位置
    '''
    if p==0: return 1
    p_out = list(dep_T2H.get(p,{}).values())
    p_out = list(chain(*p_out))
    if not p_out or min(p_out)>p: return p
    smallest = find_smallest(dep_T2H, min(p_out))
    return smallest

def find_biggest(dep_T2H, p):
    '''
    寻找当前中心语的修饰覆盖的范围
    由于不存在交叉的情况，所以只要往前找最小的即可
    return: 第一个词对应的位置
    '''
    p_out = list(dep_T2H.get(p,{}).values())
    p_out = list(chain(*p_out))
    if not p_out or max(p_out)<p: return p
    biggest = find_biggest(dep_T2H, max(p_out))
    return biggest

ltp = LTP('LTP/small')


In [4]:
# 把模型分词错误的地方替换为可以正确分词的内容
replace_dic = {'压强': '压力'}

ltp.add_words(words=list(replace_dic.keys()), freq=2)

def deal_miss_word(txt, replace_dic):
    '''将分词错误的地方替换为可以正确分词的内容'''
    words = ltp.pipeline([txt], tasks = ["cws"]).cws[0]
    recover_dic = {}
    for i in range(len(words)):
        if words[i] in replace_dic:
            recover_dic[i] = words[i]
            words[i] = replace_dic[words[i]]
    return ''.join(words), recover_dic

def span_contain(span1, span2):
    '''span1是否包含span2'''
    if span1[0]<=span2[0] and span1[1]>=span2[1]:
        return True
    return False

def del_intra_span(phrase_span):
    '''去掉被其他span包围的span'''
    if not phrase_span: return phrase_span
    merge_span = [phrase_span[-1]]
    for i in range(len(phrase_span)-2, -1, -1):
        cur_span = phrase_span[i]
        if not span_contain(merge_span[-1], cur_span):
        # if (cur_span[0]-merge_span[-1][0])*(merge_span[-1][1]-cur_span[1])<0:
            merge_span.append(cur_span)
    merge_span.reverse()
    return merge_span

def del_inter_span(lst1, lst2):
    '''只保留span1中没有被span2中元素完全覆盖的部分'''
    i, j = 0, 0
    len1, len2 = len(lst1), len(lst2)
    del_idx = []
    while i<len1 and j<len2:
        if span_contain(lst2[j], lst1[i]):
            del_idx.append(i)
            i += 1
            continue
        # if lst1[i][1]<=lst2[j][0]: i+=1
        if lst2[j][1]<=lst1[i][0]: j += 1
        else: i += 1
    lst = [lst1[i] for i in range(len1) if i not in del_idx]
    return lst


'''
node-edge-node
node-edge-tri
tri-edge-tri
'''
class Trip:
    def __init__(self, head, rel, tail):
        '''
        mode: 元素的表达形式
            span表示起止范围，如[2,5]；
            str表示字符串，如'灭火器'
        '''
        self.head = head
        self.rel = rel
        self.tail = tail
        # self.val = '[{}-{}-{}]'.format(
        #     head[1], rel[1], tail[1]
        # )

class Node:
    def __init__(self, start, end):
        self.start = start
        self.end = end

def have_dep(idx, tag, dep, dep_T2H):
    '''判断当前词是否做对应成分'''
    f1 = tag in dep_T2H[idx+1]
    f2 = False
    if dep[idx][2] == 'COO':
        f2 = have_dep(dep[idx][1]-1, tag, dep, dep_T2H)
    return f1 or f2

def get_all_coo(idx, dep_T2H):
    '''按照在序列中出现的顺序得到和当前词并列的所有词'''
    c1 = dep_T2H[idx+1].get('COO', [])
    c2 = []
    for c in c1:
        c2.append(c)
        c2 += get_all_coo(c-1, dep_T2H)
    return c2

In [ ]:
a = {}
b = {1:2}
a.update(b)
a

In [5]:
from KPR import get_so

class RuleNER:
    '''
    用规则系统做NER
    基本上所有以名词为中心语的短语都可以当作实体先抽出来
    '''
    def __init__(self, ltp) -> None:
        self.ltp = ltp
    
    def get_spd(self, txt):
        txt, recover_dic = deal_miss_word(txt, replace_dic)
        result = self.ltp.pipeline([txt], tasks = ["cws","dep","pos"])
        seg = result.cws[0]
        pos = result.pos[0]
        dep = result.dep[0]
        for key in recover_dic:
            seg[key] = recover_dic[key]
        dep = list(zip(range(1,1+len(seg)), dep['head'], dep['label']))
        return seg, pos, dep

    def get_full_trips(self, txt, offset=0):
        '''得到扩展三元组'''
        seg, pos, dep = self.get_spd(txt)
        dep_T2H = build_T2H(dep)   # 构建发射字典
        print('71 self.dep_T2H', dep_T2H)
        so_spans = self.get_so(seg, pos, dep, dep_T2H)
        adv_spans = self.get_adv(seg, pos, dep, dep_T2H)
        preda_spans = self.get_preda(seg, pos, dep, dep_T2H)

        idx_so_span = {}
        idx_uncon_so_span = {}
        idx_preda_span = {}
        for span in so_spans:
            idx_so_span.update({s:span for s in range(span[0], span[1]+1)})
        # for span in uncon_so_spans:
        #     idx_uncon_so_span.update({s:span for s in range(span[0], span[1]+1)})
        for span in preda_spans:
            idx_preda_span.update({s:span for s in range(span[0], span[1]+1)})
        head = list(dep_T2H[0].values())[0][0]
        hed_list = [head] + get_all_coo(head-1, dep_T2H)
        ########## 要处理解析结果中没有谓语的情况 ##########
        # 相对密度不小于0.75的可燃气体
        print('132', hed_list)
        hed_spans = [idx_preda_span[i] for i in hed_list]
        hed_spans = del_intra_span(hed_spans)
        print('135', hed_list, hed_spans)
        # hed_spans = []
        # for i in hed_list:
        #     if idx_preda_span[i] not in hed_spans:
        #         hed_spans.append(idx_preda_span[i])
        single_trips = self.get_single_trips(
            seg, pos, dep, dep_T2H, hed_spans, idx_preda_span, 
            idx_so_span, idx_uncon_so_span
        )
        
        def shift_offset(spans):
            '''递归得到字符串'''
            spans_shift = []
            for s in spans:
                # print('414', s)
                # if len(s)==3:
                if not s:
                    span_str = None
                elif s[-1]=='tri':
                    span_str = shift_offset(s[:-1])
                    span_str.append('tri')
                elif s[-1]!='node':
                    span_str = shift_offset(s)
                else:
                    span_str = [s[0]+offset, s[1]+offset]
                    span_str.append('node')
                spans_shift.append(span_str)
            return spans_shift
        
        single_trips = [
            shift_offset(single_trip) for single_trip in single_trips
        ]
        # print('154', single_trips)
        return single_trips

    def get_so(self, seg, pos, dep, dep_T2H):
        '''抽取简单主语和、宾语和谓语'''

        def is_skip_word(idx):
            '''判断当前词是否需要被跳过'''
            # 状语，设置泄压设施时，泄压部位应能在爆炸作用达到结构最大耐受压强前泄压，其泄压方向不应朝向人员聚集的场所和人行通道；
            # 不是方位名词时基本是条件状语，不适合以此为中心语提取短语
            # 在温度大于100时，加油站应设在供应站和调压站、加油站；当地铁站位于道路下方，加油站应设在供应站和调压站、加油站
            # if dep[i][2]=='POB' and pos[i]!='nd': continue
            # 在体育馆内部，体育场要打开抽湿器，关闭加热器
            # 当温度大于30，湿度大于50时，体育场要打开抽湿器，关闭加热器
            f1 = dep[idx][2]=='ADV'
            f2 = False
            f3 = False
            # f4 = idx>0 and seg[idx-1]=='的'
            if dep[idx][2]=='POB':                              # 介宾短语中宾语的修饰语有主谓结构：当温度大于30，湿度大于50时，体育场要打开抽湿器，关闭加热器
                att_idx = dep_T2H[idx+1].get('ATT', [-1])[0]
                for tag in ['SBV','FOB','VOB']:
                    if tag in dep_T2H[att_idx]:
                        f2 = True
                        break
            for tag in ['SBV','FOB','VOB']:                     # 谓语不可能是主宾语的中心词：当温度大于30，湿度大于50时，我们要保证体育场打开抽湿器，关闭加热器
                if have_dep(idx, tag, dep, dep_T2H):
                    f3 = True
                    break
            # print('160', seg[idx], f4, (f1 or f2 or f3)==((f1 or f2 or f3) and not f4))
            # print(f1 or f2 or f3, (f1 or f2 or f3) and not f4)
            # return f1 or f2 or f3
            return f1 or f2 or f3
        
        def is_anchor_word(idx):
            '''判断当前词是否可以当作锚点'''
            f1 = 'n' in pos[idx] or pos[idx] in ['m', 'q']      # 数词和量词可以当中心点：应根据本规范第2.1节进行合规性判定。
            f2 = dep[idx][2] in ['SBV','FOB','VOB']
            f3 = False
            f4 = idx>0 and seg[idx-1]=='的'                     # 前面是“的”则认为该词可以做中心语：本规范适用于新建、扩建、改建的民用与工业建筑中自动喷水灭火系统的设计。
            if dep[idx][2]=='COO':
                f3 = is_anchor_word(dep[idx][1]-1)
            return f1 or f2 or f3 or f4

        # def is_concat_word(idx, anchor_list):
        #     '''
        #     如果当前词的COO能链接到上一个span，则合并
        #     '''
        #     f1 = dep[i][2]=='COO'
        #     print('146', idx, dep[idx], anchor_list)
        #     if dep[idx][1] in anchor_list:
        #         anchor_list.append(idx+1)
        #         return True
        #     return False

        def is_concat_word(idx, pre_span):
            '''
            如果当前词的COO能链接到上一个span，则合并
            '''
            f1 = dep[idx][2]=='COO'
            f2 = pre_span and pre_span[0]<=dep[idx][1]<=pre_span[1]
            return f1 and f2

        # seg, pos, dep = self.seg, self.pos, self.dep
        # dep_T2H = self.dep_T2H
        phrase_span = [[]]
        uncon_so_spans = []
        for i in range(len(pos)):
            if is_skip_word(i): continue
            # if 'n' in pos[i] or dep[i][2] in ['SBV','FOB','VOB']:
            if is_anchor_word(i):
                start = find_smallest(dep_T2H, i+1)
                while True:                                                 # 在找当前范围的同时判断之前的是否被覆盖
                    if phrase_span[-1] and start<=phrase_span[-1][0]:
                        phrase_span.pop()
                        uncon_so_spans.pop()
                    else:
                        break
                uncon_so_spans.append([start, i+1])
                if is_concat_word(i, phrase_span[-1]):
                    start = phrase_span[-1][0]
                    phrase_span.pop()
                phrase_span.append([start, i+1])
        phrase_span = phrase_span[1:]
        # print('phrase_span', [seg[s[0]-1:s[1]] for s in phrase_span])
        # merge_span = del_intra_span(phrase_span)
        phrases = [seg[s[0]-1:s[1]] for s in phrase_span]
        print('noun phrases', phrases)
        # print(uncon_so_spans)
        # uncon_phrases = [seg[s[0]-1:s[1]] for s in uncon_so_spans]
        # print('uncon noun phrases', uncon_phrases)
        # return phrase_span, uncon_so_spans
        return phrase_span
    
    def get_adv(self, seg, pos, dep, dep_T2H):
        '''
        抽取状语
        状中结构，从中心语向前找
        介宾结构，从介词向后找
        '''
        # seg, pos, dep = self.seg, self.pos, self.dep
        # dep_T2H = self.dep_T2H
        phrase_span = []
        for i in range(len(pos)):
            if dep[i][2]=='ADV':
                # 状中结构ADV确定状语结束位置：当温度大于30，湿度大于50时，体育场要打开抽湿器，关闭加热器
                start = find_smallest(dep_T2H, i+1)
                end = dep_T2H[i+1].get('POB', [i+1])[0]
                phrase_span.append([start, end])
            elif dep[i][2]=='POB' and dep[dep[i][1]-1]=='ADV':
                # 介宾短语POB确定状语的结束位置：甲、乙类工厂和仓库应设在可燃气体充装站、供应站和调压站、加油站
                phrase_span.append([dep[i][1], i+1])
        merge_span = del_intra_span(phrase_span)
        phrases = [seg[s[0]-1:s[1]] for s in merge_span]
        print('adv phrases', phrases)
        return merge_span

    def get_preda(self, seg, pos, dep, dep_T2H):
        '''
        像SO一样抽取所有的谓词，在三元组抽取中，用HED及其并列位置借助字典进行映射
        抽取谓词，没有处理主语从句、定语从句的谓词
        句子的核心词及其并列词，且是动词或介词
        是动词时需要将补语算作谓词的一部分
        介词做谓语：仓库应在居住区外部
        补语结构：仓库应设在居住区外部
        '''
        # 0: {'HED': [20]}
        def is_concat_preda(i, pre_span):
            '''
            谓词合并需要满足三个条件
            f1：连词直接连到上一个span后面
            f2：当前谓词跟上一个span的谓词是并列关系
            f3：当前谓词没有直接相连的FOB或SBV
            样例：
                仓库应根据储存物质的性质和储存物质中可燃物的数量等因素确定和规划防火要求，并应符合下列规定
                当温度大于或明显等于30，湿度大于50时，体育场要打开抽湿器，关闭加热器
            '''
            # f1 = seg[i-2]=='、' or dep_T2H[i].get('LAD', [-2])[0]==i-1
            # print('245', i, dep[i-1], pre_span, dep[i-1][1]==pre_span[0])
            # f1 = seg[i-2]=='、' or dep_T2H[i].get('LAD', [-2])[0]==pre_span[-1]+1
            # f2 = dep[i-1][1]==pre_span[0]
            # f3 = 'SBV' not in dep_T2H[i] and 'FOB' not in dep_T2H[i]
            # return i>1 and f1 and f2 and f3
            if not pre_span: return False
            f1 = seg[i-1]=='、' or dep_T2H[i+1].get('LAD', [-2])[0]==pre_span[-1]+1
            f1 = f1 or dep[i][0]-dep[i][1]==1                                           # 跟上一个动词连续并列：在建筑防火中贯彻执行国家技术经济政策
            f2 = dep[i][1]==pre_span[0]
            f3 = 'SBV' not in dep_T2H[i+1] and 'FOB' not in dep_T2H[i+1]
            return i>0 and f1 and f2 and f3

        phrase_span = [[]]
        for i in range(len(pos)):
            f1 = False
            for tag in ['SBV','FOB','VOB']:
                if have_dep(i, tag, dep, dep_T2H):
                    f1 = True
                    break
            f2 = pos[i]!='a'
            f3 = pos[i]=='a' and dep_T2H[i+1].get('SBV',[-1])[0]!=i
            if f1 and (f2 or f3):
                start = i + 1
                # 动词带有补语其表意才完整：甲、乙类工厂和仓库应设在可燃气体充装站、供应站和调压站、加油站
                end = dep_T2H.get(i+1).get('CMP', [i+1])[0]
                if dep[i][2]=='COO' and is_concat_preda(i, phrase_span[-1]):
                    start = phrase_span[-1][0]
                    phrase_span.pop()
                phrase_span.append([start, end])
        phrase_span = phrase_span[1:]

        # phrase_span = []
        # head = dep_T2H[0]['HED'][0]
        # # preda_list = [head] + dep_T2H.get(head,{}).get('COO',[])
        # preda_list = [head] + get_all_coo(head-1, dep_T2H)
        # for i in preda_list:
        #     if pos[i-1] in ['v','p']:
        #         start = i
        #         # 动词带有补语其表意才完整：甲、乙类工厂和仓库应设在可燃气体充装站、供应站和调压站、加油站
        #         end = dep_T2H.get(i, {}).get('CMP', [i])[0]
        #         if dep[i-1][2]=='COO' and is_concat_preda(i, phrase_span[-1]):
        #             start = phrase_span[-1][0]
        #             phrase_span.pop()
        #         phrase_span.append([start, end])

        phrases = [seg[s[0]-1:s[1]] for s in phrase_span]
        print('preda phrases', phrases)
        # merge_span = del_intra_span(phrase_span)
        # phrases = [seg[s[0]-1:s[1]] for s in merge_span]
        # print('preda merge_span', phrases)
        return phrase_span
    
    def bi_subj(self, preda, seg, pos):
        '''
        处理双主语结构的句子
        看似没有宾语，但实际上是两个主语发生这个动作
        也会出现在定语从句、主语从句中
        满足判定条件则返回双主语构成的三元组
            谓语在句子最后，且要对谓词分类，把谓词出现在句尾的拿出来，标一下分类数据
            连词和谓语之间是一个完整的名词短语
            连词之前的词属于一个名词短语
        样例：
        瓶装液化石油气不应跟其他化学危险物品混放；
        生产过程中散发的可燃气体、蒸气、粉尘或纤维与供暖管道、散热器表面接触能引起燃烧的场所
        生产过程中散发的可燃气体、蒸气、粉尘或纤维与供暖管道、散热器表面接触能引起燃烧
        生产过程中散发的可燃气体、蒸气、粉尘或纤维与供暖管道、散热器表面接触    短语的合并需要打补丁
        '''
        ########## 需要根据谓词类别判断是双主语还是单主语 ##########
        f1 = preda[-1]==len(pos) or pos[preda[-1]]=='wp'        # 谓语在句子最后
        f2 = False
        f3 = False
        for i in range(preda[0]-1, -1, -1):
            if seg[i] in '与和跟': break
        if i>0:
            # 连词和谓语之间是一个完整的名词短语
            txt = ''.join(seg[i+1:preda[0]-1])
            seg1, pos1, dep1 = self.get_spd(txt)
            dep_T2H1 = build_T2H(dep1)   # 构建发射字典
            so_spans1 = self.get_so(seg1, pos1, dep1, dep_T2H1)
            if so_spans1 and so_spans1[0][1]-so_spans1[0][0]==preda[0]-i-3:
                so = [i+2, preda[0]-1, 'node']
                f2=True
            # 连词之前的词属于一个名词短语
            txt = ''.join(seg[:i])
            seg1, pos1, dep1 = self.get_spd(txt)
            dep_T2H1 = build_T2H(dep1)   # 构建发射字典
            so_spans1 = self.get_so(seg1, pos1, dep1, dep_T2H1)
            if so_spans1 and so_spans1[-1][-1]==i: 
                pre_so = so_spans1[-1] + ['node']
                f3=True
        
        if f1 and f2 and f3: return True, [pre_so], [so]
        return False, [], []

        # so = idx_uncon_so_span.get(preda[0]-1, [0]).copy()
        # pre_so = idx_uncon_so_span.get(so[0]-1, [])
        # f1 = preda[-1]==len(pos) or pos[preda[-1]]=='wp'        # 谓语在句子最后
        # f2 = so[-1] and seg[so[0]-1] in '与和跟'                # 谓语的前一个词是节点短语且第一个词是连词
        # f3 = len(pre_so)                                        # 节点短语前面还是一个节点短语
        # if f1 and f2 and f3:
        #     so[0] += 1
        #     pre_so.append('node')
        #     so.append('node')
        #     return True, [pre_so], [so]
        # return False, [], []


    def get_single_trips(
        self, seg, pos, dep, dep_T2H, hed_spans, idx_preda_span,
        idx_so_span, idx_uncon_so_span
    ):
        '''
        针对每一个中心谓语，提取其三元组及嵌套的结果
        得到简单三元组，解决了宾语从句的表示问题
        根据主谓宾关系得到三元组、二元组
        '''
        # seg, pos, dep = self.seg, self.pos, self.dep
        # dep_T2H = self.dep_T2H
        # preda_spans = self.preda_spans
        # idx_so_span = self.idx_so_span
        
        def get_head_anchor(s):
            '''得到头节点的锚点'''
            anchor = -1
            if 'SBV' in dep_T2H[s]:
                anchor = dep_T2H[s]['SBV'][0]
            elif 'FOB' in dep_T2H[s]:
                anchor = dep_T2H[s]['FOB'][0]
            elif dep[s-1][2] == 'COO':
                anchor = get_head_anchor(dep[s-1][1])
            return anchor
        
        def get_tail_anchor(s):
            '''
            得到尾节点的锚点
            有动宾结构和介宾结构两种可能
            '''
            anchor = -1
            if 'VOB' in dep_T2H[s]:
                anchor = dep_T2H[s]['VOB'][0]
            elif pos[s-1]=='p':
                anchor = dep_T2H[s].get('POB', [-1])[0]
            elif s<len(pos) and pos[s]=='p' and dep[s][1]==s:
                anchor = dep_T2H[s+1].get('POB', [-1])[0]
            return anchor

        def preda_of_clause(anchor):
            '''判断当前词是否为从句的谓词'''
            f1 = pos[anchor-1] in ['v','p']
            f2 = False
            for tag in ['SBV','FOB','VOB']:
                if have_dep(anchor-1, tag, dep, dep_T2H):
                    f2 = True
                    break
            return f1 and f2

        def get_node(anchor):
            '''
            根据anchor得到节点范围
            谓词有COO则直接拆分为多个阶段
            非谓词有COO则将其范围内连续的COO合成一个
            '''
            if anchor == -1:
                return [None]
            elif preda_of_clause(anchor):
                # 当前中心语是宾语从句的谓词，则返回三元组列表
                # anchor_list = [anchor] + get_all_coo(anchor-1, dep_T2H)
                # node_list = []
                # node_list = [get_recur_trip(idx_preda_span[a]) for a in anchor_list]
                # node_list = [node.append('tri') for node in node_list]
                # for a in anchor_list:
                    # node_list += get_recur_trip(idx_preda_span[a])
                    
                # 规范树解析出错，需要分级解析：本规范要预防建筑火灾和减少火灾危害，确保生命财产的安全，并在建筑防火中贯彻执行国家技术经济政策，确保建筑的防火符合安全可靠、经济合理、技术先进、确保质量的要求
                # 不用再管并列，把从句当主句时自会处理并列：当温度大于30，湿度大于50时，我们要保证体育场打开抽湿器，关闭加热器
                start = find_smallest(dep_T2H, anchor)
                end = find_biggest(dep_T2H, anchor)
                txt = ''.join(seg[start-1:end])
                # print('403', start, txt)
                node_list = self.get_full_trips(txt, start-1)
                for n in node_list: n.append('tri')
                # node = get_recur_trip(idx_preda_span[anchor])
                # 有COO则返回多个trip列表，统一格式，都返回node的列表
                # 谓词也学SO搞个idx到span的映射，处理并列的情况
                # return node
                return node_list
            else:
                # start = find_smallest(dep_T2H, anchor)
                # node = Node(start, anchor)
                # node = [start, anchor]
                node = idx_so_span.get(anchor, None)
                if node[-1]!='node': node.append('node')
                return [node]

        def get_recur_trip(preda):
            '''
            得到带有递归节点的三元组
            anchor有COO：甲、乙类工厂和仓库应设在可燃气体充装站、供应站和调压站、加油站
            从句有COO：建筑的防火应符合下列目标要求：保障人身生命和财产安全、人身健康；
            '''
            flag, head, tail = self.bi_subj(preda, seg, pos)

            if not flag:
                anchor = get_head_anchor(preda[0])
                head = get_node(anchor)

                for p in preda:                         # 存在补语加VOB的情况：我们应看完饭、吃完饭、做完饭
                    anchor = get_tail_anchor(p)
                    if anchor!=-1: break
                tail = get_node(anchor)

            # 有多个head或tail则做笛卡尔积
            if preda[-1]!='node': preda.append('node')
            trip_list = []
            '''
            如果head和tail是node则要解析其定语
                有谓语则解析句子，没谓语则当作节点定语
            针对preda解析其状语，句子以句号为分隔
                有谓语则解析句子，没谓语则当节点状语
            可以先不处理定语
            基本节点的格式为：{
                trip: [h, preda, t],
                modi_h: [node, cell],
                modi_t: [node, cell],
                adv: [node, cell]
            }
            '''
            adv_list = []
            for adv_anchor in dep_T2H:
                pass
            for h in head:
                for t in tail:
                    trip_list.append([h,preda,t])
            return trip_list


        # def get_node(anchor):
        #     '''
        #     根据anchor得到节点范围
        #     谓词有COO则直接拆分为多个阶段
        #     非谓词有COO则将其范围内连续的COO合成一个
        #     '''
        #     if anchor == -1:
        #         return None
        #     elif preda_of_clause(anchor):
        #         # 当前中心语是宾语从句的谓词，则返回三元组列表
        #         anchor_list = [anchor] + get_all_coo(anchor-1, dep_T2H)
        #         node_list = [get_recur_trip(idx_preda_span[a]) for a in anchor_list]
        #         # node_list = [node.append('tri') for node in node_list]
        #         for n in node_list: n.append('tri')
        #         print('365', node_list)
        #         node = get_recur_trip(idx_preda_span[anchor])
        #         # 有COO则返回多个trip列表，统一格式，都返回node的列表
        #         # 谓词也学SO搞个idx到span的映射，处理并列的情况
        #         # return node
        #         return node_list
        #     else:
        #         # start = find_smallest(dep_T2H, anchor)
        #         # node = Node(start, anchor)
        #         # node = [start, anchor]
        #         node = idx_so_span.get(anchor, None)
        #         if node[-1]!='node': node.append('node')
        #         return node

        # def get_recur_trip(preda):
        #     '''
        #     得到带有递归节点的三元组
        #     anchor有COO：甲、乙类工厂和仓库应设在可燃气体充装站、供应站和调压站、加油站
        #     从句有COO：建筑的防火应符合下列目标要求：保障人身生命和财产安全、人身健康；
        #     '''
        #     anchor = get_head_anchor(preda[0])
        #     head = get_node(anchor)

        #     for p in preda:                         # 存在补语加VOB的情况：我们应看完饭、吃完饭、做完饭
        #         anchor = get_tail_anchor(p)
        #         if anchor!=-1: break
        #     tail = get_node(anchor)

        #     # 有多个head或tail则做笛卡尔积
        #     if preda[-1]!='node': preda.append('node')
        #     return [head, preda, tail]

        # def get_str(spans):
        #     '''递归得到字符串'''
        #     spans_str = []
        #     for s in spans:
        #         # print('414', s)
        #         # if len(s)==3:
        #         if s[-1]!='node':
        #             span_str = get_str(s)
        #         else:
        #             span_str = seg[s[0]-1:s[1]] if s else []
        #         spans_str.append(span_str)
        #     return spans_str

        trip_spans = []
        print('510', hed_spans)
        for preda in hed_spans:
            '''
            根据谓词判断三元组，借助COO处理省略主语的情况
            头实体：先找SBV、FOB，没有则顺着COO找，没有则用上级条文的主语
            尾实体：找VOB，没有则记占位节点
            仓库的防火要求应根据储存物质的性质和储存物质中可燃物的数量等因素确定，并应符合下列规定
            连词相连的谓词要先合并再拆解
            仓库应根据储存物质的性质和储存物质中可燃物的数量等因素确定和规划防火要求，并应符合下列规定
            如果anchor处是从句的谓语，则递归执行该函数
            '''
            trip_spans += get_recur_trip(preda)
        
        '''在这里给'''
        print(trip_spans)

        def get_str(spans):
            '''递归得到字符串'''
            spans_str = []
            for s in spans:
                # print('414', s)
                # if len(s)==3:
                if not s:
                    span_str = []
                elif s[-1]=='tri':
                    span_str = get_str(s[:-1])
                elif s[-1]!='node':
                    span_str = get_str(s)
                else:
                    span_str = seg[s[0]-1:s[1]]
                spans_str.append(span_str)
            return spans_str
        
        phrases = [
            get_str(trip_span) for trip_span in trip_spans
        ]
        print('single trip phrases', phrases)
        print(trip_spans)
        return trip_spans

    
    def deal_digestion(self):
        '''
        确定其的指代内容
        做三元组成分，则替换为对应的名词短语
        做修饰语，则替换为名词短语+的
        指代判定依据：前一个名词短语或三元组的头
            所有最基本的头尾节点都要经过指代消解
        做修饰语：陆上消防站应位于易燃易爆危险品场所或设施的常年主导风向的上风或侧风处，其用地边界距离甲、乙类厂房、加油、加气站及易燃易爆危险品储存场所不应小于50m；
        做三元组成分：建筑承重结构应保证其在受到火或高温作用后仍能在设计耐火时间内正常发挥功能；
        远距离指代：设置泄压设施时，泄压部位应能在爆炸作用达到结构最大耐受压强前泄压，其泄压方向不应朝向人员聚集的场所和人行通道
        '''
        
    def deal_attribute():
        '''
        处理开头是介词的，之间、间距、距离
        短语开头是介词：消防站执勤车辆的主出入口距离大型人员密集的公共建筑的主要疏散出口不应小于50m。
        短语结尾是之间、间距、距离：甲、乙类工厂和仓库，可燃气体充装站、供应站和调压站，汽车加油加气站等之间及与其他建筑的间距，应符合消防安全要求；
        短语开头是介词，结尾是间距：易燃易爆危险品库房与在建工程的防火间距不应小于15m，与固定动作作业区不应小于12m，与邻近人员密集区、建筑物相对集中区及其他建筑的间距应符合消防要求；
        省略属性，属性值的头实体应该是属性
        '''

    def split_phrase():
        '''
        分割名词短语
        把部位当作属性节点
        先把逗号分开，再按照一般拆解过程来
        '''

    def get_modi():
        '''
        定语从句，谓词是ATT的尾，向前向后找全覆盖
        中心语用连续ATT和ADV确定范围
        面积大于100平米的消防水泵配电应能在火灾时保持不间断供电，其线路应为专用消防配电线路。
        实体的定语还是在关系节点上，内边上不附着定语或状语
        '''
    
    def get_adv_trip():
        '''解析状语从句'''
    
    def get_modi_trip():
        '''解析定语从句'''

'''
把“压强”换成“压力”，解析完后再换回来
词典缺失词替换
提取带定语的名词短语
提取谓词、状语
提取三元组，状语内的名词短语不参与三元组构建
嵌套提取主语从句、宾语从句中的三元组

名词短语处理：
共指消解，其
处理双头属性，开头是介词的，之间、间距、距离

名词短语解析，提取定语，定语分配给各个并列中心语
定语分为修饰、从句、实体
解析定语从句
中心语分类，哪些是实体，哪些是属性
解析一般实体，实体-包含-实体
解析一般属性，实体-内边-属性
处理省略实体、省略属性的属性，属性值之前必是属性
拆解并列中心语
'''

rule_ner = RuleNER(ltp)
txt = '建筑的选址和总平面布局应符合减小火灾危害，方便灭火救援的要求，并应符合下列规定'
# txt = '生产和储存易燃易爆物品的工厂、仓库等应位于城镇规划区的边缘或相对独立的安全地带；'
# txt = '储罐区的低倍数泡沫灭火系统应符合下列规定：对于非水溶性甲、乙、丙类液体固定顶储罐，应为液上喷射、液下喷射或半液下喷射系统；'
txt = '甲、乙类工厂和仓库，可燃气体充装站、供应站和调压站，汽车加油加气站等之间及和其他建筑的间距，应符合消防安全要求；'
# txt = '消防站执勤车辆的主出入口距离大型人员密集的公共建筑的主要疏散出口不应小于50m。'
'''如果开头词性是p，则有可能表示属性，实体末尾是之间、距离，则要当相对属性来处理'''
txt = '陆上消防站应位于易燃易爆危险品场所或设施的常年主导风向的上风或侧风处，其用地边界距离甲、乙类厂房、加油、加气站及易燃易爆危险品储存场所不应小于50m；'
txt = '建筑内的防火分隔应能在其设计耐火时间内阻止火势与烟气蔓延至其他区域'
txt = '设置泄压设施时，泄压部位应能在爆炸作用达到结构最大耐受压强前泄压，其泄压方向不应朝向人员聚集的场所和人行通道；'
# txt = '除粮食等筒仓外，无法设置泄压设施或泄压面积不符合要求时，相应部位的建筑承重结构和防火分隔结构应满足抗爆要求。'
'''有没有介词会很影响状语的抽取规则'''
txt = '当仓库足够大时，仓库的防火要求应根据储存物质的性质和储存物质中可燃物的数量等因素确定，并应符合下列规定：'
txt = '仓库应根据储存物质的性质和储存物质中可燃物的数量等因素确定和规划防火要求，并应符合下列规定'
# txt = '仓库不应设在居住区内部'
txt = '保障施工现场消防供水的消防水泵配电应能在火灾时保持不间断供电，其线路应为专用消防配电线路。'
# txt = '戊类仓库的防火要求'
# txt = '建筑承重结构应保证其在受到火或高温作用后仍能在设计耐火时间内正常发挥功能'
txt = '甲、乙类工厂和仓库应设在可燃气体充装站、供应站和调压站、加油站'
txt = '当温度大于或明显等于30，湿度大于50时，体育场要打开抽湿器，关闭加热器'
# txt = '温度大于30，湿度大于50'
# txt = '当温度大于30或高度明显等于30，湿度大于50时，体育场要打开抽湿器，关闭加热器'
# txt = '建筑的防火应符合下列目标要求：保障生命财产安全和身体健康；'
txt = '当温度大于30，湿度大于50时，我们要保证体育场打开抽湿器，关闭加热器'
# txt = '仓库应根据储存物质的性质和储存物质中可燃物的数量等因素确定和规划防火要求，并应符合下列规定'
# txt = '为了预防建筑火灾和减少火灾危害，确保生命财产的安全，并在建筑防火中贯彻执行国家技术经济政策，确保建筑的防火符合安全可靠、经济合理、技术先进、确保质量的要求，依据有关法律、法规，制定本规范。'
# txt = '新建、改建和扩建建筑在规划、设计、施工与使用中的防火技术与措施，应遵守本规范。' # 有两个SBV
txt = '当建筑防火中采用的方法、技术、材料与制品、措施等与本规范的规定不同或有特殊要求时，应根据本规范第2.1节进行合规性判定。'
txt = '当方法与制品等与规定不同或有特殊要求时，应根据本规范第2.1节进行合规性判定。'
# txt = '建筑防火中采用的方法、技术、材料与制品、措施等与本规范的规定不同或有特殊要求'
# txt = '无障碍设施应保证安全和各类人群的方便使用。'
# txt = '本规范适用于新建、扩建、改建的民用与工业建筑中自动喷水灭火系统的设计。'
# txt = '本规范要预防建筑火灾和减少火灾危害，确保生命财产的安全，并在建筑防火中贯彻执行国家技术经济政策，确保建筑的防火符合安全可靠、经济合理、技术先进、确保质量的要求'
# txt = '建筑中散发较空气重的可燃气体、蒸气或有粉尘、纤维爆炸危险的场所或部位应符合下列规定'
# txt = '瓶装液化石油气不应与其他化学危险物品混放；'
# txt = '生产过程中散发的可燃气体、蒸气、粉尘或纤维与供暖管道、散热器表面接触'
# txt = '新建、改建或拆除建筑、结构、设备或类似活动所用临时电气线路和照明器具、涉及施工所需的易燃和可燃物质的使用与存放以及施工现场的用火、用电、用气应符合消防安全要求。'
# txt = '地下、半地下场所内不应使用或储存液化石油气、相对密度不小于0.75的可燃气体、闪点低于60℃的液体燃料，不应有相应的燃气或可燃液体配送管道。'
# txt = '地下、半地下场所内不应使用或储存液化石油气、可燃气体、液体燃料，不应有燃气或可燃液体配送管道。'
# txt = '相对密度不小于0.75的可燃气体'
txt = '施工临时办公与生活用房、发电机房、变配电站、厨房操作间、锅炉房和可燃材料与易燃易爆物品库房，当采用金属夹芯板材时，其芯材的燃烧性能应为A级。'
res = rule_ner.get_spd(txt)
word_pos = list(zip(res[0], res[1], res[2]))
print(word_pos)
# print(res[2])
rule_ner.get_full_trips(txt)


[('施工', 'v', (1, 0, 'HED')), ('临时', 'b', (2, 3, 'ADV')), ('办公', 'v', (3, 23, 'ATT')), ('与', 'c', (4, 6, 'LAD')), ('生活', 'v', (5, 6, 'ATT')), ('用房', 'n', (6, 3, 'COO')), ('、', 'wp', (7, 9, 'WP')), ('发电机', 'n', (8, 9, 'ATT')), ('房', 'n', (9, 3, 'COO')), ('、', 'wp', (10, 9, 'WP')), ('变配电站', 'v', (11, 9, 'ATT')), ('、', 'wp', (12, 19, 'WP')), ('厨房', 'n', (13, 14, 'ATT')), ('操作间', 'n', (14, 19, 'ATT')), ('、', 'wp', (15, 16, 'WP')), ('锅炉房', 'n', (16, 19, 'COO')), ('和', 'c', (17, 19, 'LAD')), ('可燃', 'i', (18, 19, 'ATT')), ('材料', 'n', (19, 3, 'COO')), ('与', 'c', (20, 22, 'LAD')), ('易燃易爆', 'i', (21, 22, 'ATT')), ('物品', 'n', (22, 19, 'COO')), ('库房', 'n', (23, 1, 'VOB')), ('，', 'wp', (24, 1, 'WP')), ('当', 'p', (25, 38, 'ADV')), ('采用', 'v', (26, 30, 'ATT')), ('金属', 'n', (27, 28, 'ATT')), ('夹芯', 'v', (28, 29, 'ATT')), ('板材', 'n', (29, 26, 'VOB')), ('时', 'n', (30, 25, 'POB')), ('，', 'wp', (31, 25, 'WP')), ('其', 'r', (32, 33, 'ATT')), ('芯材', 'n', (33, 36, 'ATT')), ('的', 'u', (34, 33, 'RAD')), ('燃烧', '

[[None, [1, 1, 'node'], [2, 23, 'node']],
 [[32, 36, 'node'], [38, 38, 'node'], [39, 40, 'node']]]

In [ ]:
# 把模型分词错误的地方替换为可以正确分词的内容
replace_dic = {'压强': '压力'}

ltp.add_words(words=list(replace_dic.keys()), freq=2)

def deal_miss_word(txt, replace_dic):
    words = ltp.pipeline([txt], tasks = ["cws"]).cws[0]
    recover_dic = {}
    for i in range(len(words)):
        if words[i] in replace_dic:
            recover_dic[i] = words[i]
            words[i] = replace_dic[words[i]]
    return ''.join(words), recover_dic

def span_contain(span1, span2):
    '''span1是否包含span2'''
    if span1[0]<=span2[0] and span1[1]>=span2[1]:
        return True
    return False

def del_intra_span(phrase_span):
    '''去掉被其他span包围的span'''
    merge_span = [phrase_span[-1]]
    for i in range(len(phrase_span)-2, -1, -1):
        cur_span = phrase_span[i]
        if not span_contain(merge_span[-1], cur_span):
        # if (cur_span[0]-merge_span[-1][0])*(merge_span[-1][1]-cur_span[1])<0:
            merge_span.append(cur_span)
    merge_span.reverse()
    return merge_span

def del_inter_span(lst1, lst2):
    '''只保留span1中没有被span2中元素完全覆盖的部分'''
    i, j = 0, 0
    len1, len2 = len(lst1), len(lst2)
    del_idx = []
    while i<len1 and j<len2:
        if span_contain(lst2[j], lst1[i]):
            del_idx.append(i)
            i += 1
            continue
        # if lst1[i][1]<=lst2[j][0]: i+=1
        if lst2[j][1]<=lst1[i][0]: j += 1
        else: i += 1
    lst = [lst1[i] for i in range(len1) if i not in del_idx]
    return lst

# def deal_bi_

'''
node-edge-node
node-edge-tri
tri-edge-tri
'''
class Trip:
    def __init__(self, head, edge, tail):
        '''
        mode: 元素的表达形式
            span表示起止范围，如[2,5]；
            str表示字符串，如'灭火器'
        '''
        self.head = head
        self.edge = edge
        self.tail = tail
        self.val = '[{}-{}-{}]'.format(
            head[1], edge[1], tail[1]
        )

class Node:
    def __init__(self, start, end):
        self.start = start
        self.end = end

class RuleNER:
    '''
    用规则系统做NER
    基本上所有以名词为中心语的短语都可以当作实体先抽出来
    '''
    def __init__(self, ltp) -> None:
        self.ltp = ltp
    
    def get_spd(self, txt):
        txt, recover_dic = deal_miss_word(txt, replace_dic)
        result = self.ltp.pipeline([txt], tasks = ["cws","dep","pos"])
        seg = result.cws[0]
        pos = result.pos[0]
        dep = result.dep[0]
        for key in recover_dic:
            seg[key] = recover_dic[key]
        dep = list(zip(range(1,1+len(seg)), dep['head'], dep['label']))
        return seg, pos, dep

    def get_full_trips(self, txt):
        '''得到扩展三元组'''
        seg, pos, dep = self.get_spd(txt)
        self.seg, self.pos, self.dep = seg, pos, dep
        self.dep_T2H = build_T2H(dep)   # 构建发射字典
        print('71 self.dep_T2H', self.dep_T2H)
        self.noun_spans = self.get_noun()
        self.adv_spans = self.get_adv()
        self.preda_spans = self.get_preda()
        self.get_single_trips()

    def get_noun(self):
        '''抽取以名词为中心语的短语'''
        seg, pos, dep = self.seg, self.pos, self.dep
        phrase_span = []
        for i in range(len(pos)):
            # 状语，设置泄压设施时，泄压部位应能在爆炸作用达到结构最大耐受压强前泄压，其泄压方向不应朝向人员聚集的场所和人行通道；
            if dep[i][2]=='ADV': continue
            # 不是方位名词时基本是条件状语，不适合以此为中心语提取短语
            if dep[i][2]=='POB' and pos[i]!='nd': continue
            if 'n' in pos[i] or dep[i][2] in ['SBV','FOB','VOB']:
                start = find_smallest(self.dep_T2H, i+1)
                phrase_span.append([start, i+1])
        if not len(phrase_span): return phrase_span
        merge_span = del_intra_span(phrase_span)
        phrases = [seg[s[0]-1:s[1]] for s in merge_span]
        print('noun phrases', phrases)
        return merge_span
    
    def get_adv(self):
        '''
        抽取状语
        状中结构，从中心语向前找
        介宾结构，从介词向后找
        '''
        seg, pos, dep = self.seg, self.pos, self.dep
        phrase_span = []
        for i in range(len(pos)):
            if dep[i][2]=='ADV':
                # 状中结构ADV确定状语结束位置
                start = find_smallest(self.dep_T2H, i+1)
                phrase_span.append([start, i+1])
            elif dep[i][2]=='POB':
                # 介宾短语POB确定状语的结束位置
                phrase_span.append([dep[i][1], i+1])
        if not len(phrase_span): return phrase_span
        merge_span = del_intra_span(phrase_span)
        phrases = [seg[s[0]-1:s[1]] for s in merge_span]
        print('adv phrases', phrases)
        return merge_span

    def get_preda(self, preda_list=[]):
        '''
        抽取谓词，没有处理主语从句、定语从句的谓词
        句子的核心词及其并列词，且是动词或介词
        是动词时需要将补语算作谓词的一部分
        介词做谓语：仓库应在居住区外部
        补语结构：仓库应设在居住区外部
        '''
        # 0: {'HED': [20]}
        def coo_preda(i, pre_span):
            '''
            谓词并列需要满足三个条件
            前一个词是表示并列的
            跟上个span的第一个词是COO
            跟上个span的最后一个词只隔一个词
            样例：仓库应根据储存物质的性质和储存物质中可燃物的数量等因素确定和规划防火要求，并应符合下列规定
            '''
            f1 = seg[i-2]=='、' or dep_T2H[i].get('LAD', [-2])[0]==i-1
            f2 = dep[i-1][1]==pre_span[0]
            f3 = i - pre_span[-1]==2
            return i>1 and f1 and f2 and f3

        seg, pos, dep = self.seg, self.pos, self.dep
        dep_T2H = self.dep_T2H
        phrase_span = []
        head = dep_T2H[0]['HED'][0]
        if not preda_list:
            preda_list = [head] + dep_T2H.get(head,{}).get('COO',[])
        for i in preda_list:
            if pos[i-1] in ['v','p']:
                start = i
                # 动词带有补语其表意才完整：甲、乙类工厂和仓库应设在可燃气体充装站、供应站和调压站、加油站
                end = dep_T2H.get(i, {}).get('CMP', [i])[0]
                # print(seg[i-1], dep[i-1][2])
                if dep[i-1][2]=='COO' and coo_preda(i, phrase_span[-1]):
                    start = phrase_span[-1][0]
                    phrase_span.pop()
                phrase_span.append([start, end])

        if not len(phrase_span): return phrase_span
        merge_span = phrase_span
        phrases = [seg[s[0]-1:s[1]] for s in merge_span]
        print('preda phrases', phrases)
        return merge_span

    # def get_

    def get_single_trips(self):
        '''
        针对每一个中心谓语，提取其三元组及嵌套的结果
        得到简单三元组，解决了宾语从句的表示问题
        根据主谓宾关系得到三元组、二元组
        '''
        seg, pos, dep = self.seg, self.pos, self.dep
        dep_T2H = self.dep_T2H
        preda_spans = self.preda_spans
        
        def get_head_anchor(s):
            '''得到头节点的锚点'''
            anchor = -1
            if 'SBV' in dep_T2H[s]:
                anchor = dep_T2H[s]['SBV'][0]
            elif 'FOB' in dep_T2H[s]:
                anchor = dep_T2H[s]['FOB'][0]
            elif dep[s-1][2] == 'COO':
                anchor = get_head_anchor(dep[s-1][1])
            return anchor
        
        def get_tail_anchor(s):
            '''
            得到尾节点的锚点
            有动宾结构和介宾结构两种可能
            '''
            anchor = -1
            if 'VOB' in dep_T2H[s]:
                anchor = dep_T2H[s]['VOB'][0]
            elif pos[s-1]=='p':
                anchor = dep_T2H[s].get('POB', [-1])[0]
            elif s<len(pos) and pos[s]=='p' and dep[s][1]==s:
                anchor = dep_T2H[s+1].get('POB', [-1])[0]
            return anchor

        def preda_of_clause(anchor):
            '''判断当前词是否为从句的谓词'''
            f1 = pos[anchor-1] in ['v','p']
            f2 = False
            for syn in ['SBV','FOB','VOB']:
                if syn in dep_T2H[anchor]:
                    f2 = True
                    break
            return f1 and f2

        def get_node(anchor):
            '''
            根据anchor得到节点范围
            谓词有COO则直接拆分为多个阶段
            非谓词有COO则将其范围内连续的COO合成一个
            '''
            if anchor == -1:
                node = None
            elif preda_of_clause(anchor):
                # 当前中心语是宾语从句的谓词，则返回三元组列表
                node = get_recur_trip([anchor, anchor])
                # 有COO则返回多个trip列表，统一格式，都返回node的列表
            else:
                start = find_smallest(dep_T2H, anchor)
                node = [start, anchor]
            return node

        def get_recur_trip(preda):
            '''
            得到带有递归节点的三元组
            anchor有COO：甲、乙类工厂和仓库应设在可燃气体充装站、供应站和调压站、加油站
            从句有COO：建筑的防火应符合下列目标要求：保障人身生命和财产安全、人身健康；
            '''
            anchor = get_head_anchor(preda[0])
            head = get_node(anchor)

            anchor = get_tail_anchor(preda[0])
            tail = get_node(anchor)

            # 有多个head或tail则做笛卡尔积
            
            return [head, preda, tail]

        trip_spans = []
        for preda in preda_spans:
            '''
            根据谓词判断三元组，借助COO处理省略主语的情况
            头实体：先找SBV、FOB，没有则顺着COO找，没有则用上级条文的主语
            尾实体：找VOB，没有则记占位节点
            仓库的防火要求应根据储存物质的性质和储存物质中可燃物的数量等因素确定，并应符合下列规定
            连词相连的谓词要先合并再拆解
            仓库应根据储存物质的性质和储存物质中可燃物的数量等因素确定和规划防火要求，并应符合下列规定
            如果anchor处是从句的谓语，则递归执行该函数
            '''
            trip_spans.append(get_recur_trip(preda))
        
        def get_str(spans):
            '''递归得到字符串'''
            spans_str = []
            for s in spans:
                if len(s)==3:
                    span_str = get_str(s)
                else:
                    span_str = seg[s[0]-1:s[1]] if s else []
                spans_str.append(span_str)
            return spans_str
        
        phrases = [
            get_str(trip_span) for trip_span in trip_spans
        ]
        print('single trip phrases', phrases)
        print(trip_spans)
        return trip_spans

    
    def deal_digestion(self):
        '''
        确定其的指代内容
        做三元组成分，则替换为对应的名词短语
        做修饰语，则替换为名词短语+的
        指代判定依据：前一个名词短语或三元组的头
            所有最基本的头尾节点都要经过指代消解
        做修饰语：陆上消防站应位于易燃易爆危险品场所或设施的常年主导风向的上风或侧风处，其用地边界距离甲、乙类厂房、加油、加气站及易燃易爆危险品储存场所不应小于50m；
        做三元组成分：建筑承重结构应保证其在受到火或高温作用后仍能在设计耐火时间内正常发挥功能；
        远距离指代：设置泄压设施时，泄压部位应能在爆炸作用达到结构最大耐受压强前泄压，其泄压方向不应朝向人员聚集的场所和人行通道
        '''
        
    def deal_attribute():
        '''
        处理开头是介词的，之间、间距、距离
        短语开头是介词：消防站执勤车辆的主出入口距离大型人员密集的公共建筑的主要疏散出口不应小于50m。
        短语结尾是之间、间距、距离：甲、乙类工厂和仓库，可燃气体充装站、供应站和调压站，汽车加油加气站等之间及与其他建筑的间距，应符合消防安全要求；
        短语开头是介词，结尾是间距：易燃易爆危险品库房与在建工程的防火间距不应小于15m，与固定动作作业区不应小于12m，与邻近人员密集区、建筑物相对集中区及其他建筑的间距应符合消防要求；
        省略属性，属性值的头实体应该是属性
        '''

    def split_phrase():
        '''
        分割名词短语
        把部位当作属性节点
        先把逗号分开，再按照一般拆解过程来
        '''

    def get_modi():
        '''
        定语从句，谓词是ATT的尾，向前向后找全覆盖
        中心语用连续ATT和ADV确定范围
        面积大于100平米的消防水泵配电应能在火灾时保持不间断供电，其线路应为专用消防配电线路。
        '''

'''
把“压强”换成“压力”，解析完后再换回来
词典缺失词替换
提取带定语的名词短语
提取谓词、状语
提取三元组，状语内的名词短语不参与三元组构建
嵌套提取主语从句、宾语从句中的三元组

名词短语处理：
共指消解，其
处理双头属性，开头是介词的，之间、间距、距离

名词短语解析，提取定语，定语分配给各个并列中心语
定语分为修饰、从句、实体
解析定语从句
中心语分类，哪些是实体，哪些是属性
解析一般实体，实体-包含-实体
解析一般属性，实体-内边-属性
处理省略实体、省略属性的属性，属性值之前必是属性
拆解并列中心语
'''

rule_ner = RuleNER(ltp)
txt = '建筑的选址和总平面布局应符合减小火灾危害，方便灭火救援的要求，并应符合下列规定'
# txt = '生产和储存易燃易爆物品的工厂、仓库等应位于城镇规划区的边缘或相对独立的安全地带；'
# txt = '储罐区的低倍数泡沫灭火系统应符合下列规定：对于非水溶性甲、乙、丙类液体固定顶储罐，应为液上喷射、液下喷射或半液下喷射系统；'
txt = '甲、乙类工厂和仓库，可燃气体充装站、供应站和调压站，汽车加油加气站等之间及和其他建筑的间距，应符合消防安全要求；'
# txt = '消防站执勤车辆的主出入口距离大型人员密集的公共建筑的主要疏散出口不应小于50m。'
'''如果开头词性是p，则有可能表示属性，实体末尾是之间、距离，则要当相对属性来处理'''
txt = '陆上消防站应位于易燃易爆危险品场所或设施的常年主导风向的上风或侧风处，其用地边界距离甲、乙类厂房、加油、加气站及易燃易爆危险品储存场所不应小于50m；'
txt = '建筑内的防火分隔应能在其设计耐火时间内阻止火势与烟气蔓延至其他区域'
txt = '设置泄压设施时，泄压部位应能在爆炸作用达到结构最大耐受压强前泄压，其泄压方向不应朝向人员聚集的场所和人行通道；'
# txt = '除粮食等筒仓外，无法设置泄压设施或泄压面积不符合要求时，相应部位的建筑承重结构和防火分隔结构应满足抗爆要求。'
'''有没有介词会很影响状语的抽取规则'''
txt = '当仓库足够大时，仓库的防火要求应根据储存物质的性质和储存物质中可燃物的数量等因素确定，并应符合下列规定：'
txt = '仓库应根据储存物质的性质和储存物质中可燃物的数量等因素确定和规划防火要求，并应符合下列规定'
# txt = '仓库不应设在居住区内部'
txt = '保障施工现场消防供水的消防水泵配电应能在火灾时保持不间断供电，其线路应为专用消防配电线路。'
# txt = '戊类仓库的防火要求'
# txt = '建筑承重结构应保证其在受到火或高温作用后仍能在设计耐火时间内正常发挥功能'
txt = '甲、乙类工厂和仓库应设在可燃气体充装站、供应站和调压站、加油站'
res = rule_ner.get_spd(txt)
word_pos = list(zip(res[0], res[1], res[2]))
# print(word_pos)
# print(res[2])
rule_ner.get_full_trips(txt)
